In [1]:
import pandas as pd

# Carga del DataFrame
netflix_df = pd.read_csv('./data/netflix_titles.csv')
netflix_df.drop(columns=netflix_df.columns[12:], inplace=True)
netflix_df['date_added'] = pd.to_datetime(netflix_df['date_added'], format='mixed')

In [2]:
# Por ejemplo, un modelo de recomendación basado en contenido utilizando el género como característica
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Preprocesamiento de la columna 'listed_in' para el modelo de recomendación
tfidf = TfidfVectorizer(stop_words='english')
netflix_df['listed_in'] = netflix_df['listed_in'].fillna('')
tfidf_matrix = tfidf.fit_transform(netflix_df['listed_in'])

# Cálculo de similitud de coseno entre géneros
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)


In [3]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # Obtiene el índice del título que coincide con el título
    idx = netflix_df[netflix_df['title'] == title].index[0]

    # Obtiene las puntuaciones de similitud de coseno del título dado
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Ordena las películas según las puntuaciones de similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obtiene las 10 películas más similares
    sim_scores = sim_scores[1:11]

    # Obtiene los índices de las películas recomendadas
    movie_indices = [i[0] for i in sim_scores]

    # Devuelve los títulos de las películas recomendadas
    return netflix_df['title'].iloc[movie_indices]

# Ejemplo de recomendaciones para una película dada
recommendations = get_recommendations("Pokémon Journeys: The Series")
print(recommendations)

85      Pokémon Master Journeys: The Series
590                     Sailor Moon Crystal
1025                 Bakugan: Geogan Rising
1051                                Sonic X
1149                              Yu-Gi-Oh!
1693                    Beyblade Burst Rise
1723              Bakugan: Armored Alliance
1724                 Bakugan: Battle Planet
2136                             Zoids Wild
2739                     Pokémon the Series
Name: title, dtype: object
